### Working Demand Matrix Generation using Grid2Demand
This notebook demonstrates how to use grid2demand to build a travel demand matrix for any urban area.

In [ ]:
# 1. Setup and Imports
import os

import geopandas as gpd
import pandas as pd
import matplotlib.pyplot as plt
import osmnx as ox

import grid2demand as gd
import osm2gmns as og

import warnings
warnings.filterwarnings('ignore')

In [ ]:
# 2. Define Study Area - Define the study area using bounding box coordinates

# Study area bounding box
# Format: (min_longitude, min_latitude, max_longitude, max_latitude)
study_area_bbox = { # Manhattan
    'min_lon': -74.0479,  # West boundary
    'min_lat': 40.6829,   # South boundary
    'max_lon': -73.9067,  # East boundary
    'max_lat': 40.8820    # North boundary
}

study_area_bbox = { # Brooklyn
    'min_lon': -74.05,  # West boundary
    'min_lat': 40.56,   # South boundary
    'max_lon': -73.85,  # East boundary
    'max_lat': 40.75    # North boundary
}

print("Study Area Boundaries:")
print(f"  West:  {study_area_bbox['min_lon']:.4f}°")
print(f"  East:  {study_area_bbox['max_lon']:.4f}°")
print(f"  South: {study_area_bbox['min_lat']:.4f}°")
print(f"  North: {study_area_bbox['max_lat']:.4f}°")

# Calculate approximate dimensions
width_km = (study_area_bbox['max_lon'] - study_area_bbox['min_lon']) * 111.32 * 0.7  # rough conversion
height_km = (study_area_bbox['max_lat'] - study_area_bbox['min_lat']) * 111.32
print(f"\nApproximate dimensions:")
print(f"  Width:  {width_km:.2f} km")
print(f"  Height: {height_km:.2f} km")

In [ ]:
# 3a. PATH-A: Download and Visualize OpenStreetMap Network Data using OSMnx to download the street network

# Download street network from OpenStreetMap
print("Downloading street network from OpenStreetMap...")
print("This may take a few minutes...\n")

# Define the place name for the study area
place_name = "Brooklyn, New York City, New York, USA"  # CHANGE THIS to your study area

# Use place name to download network
# OSMnx will query the Nominatim API to get the boundaries
G = ox.graph_from_place(
    place_name,
    network_type='drive'  # Only drivable roads
)

# TODO - Clean network using OSMNx tools

print(f"✓ Network downloaded successfully!")
print(f"  Nodes (intersections): {len(G.nodes):,}")
print(f"  Edges (road segments): {len(G.edges):,}")

# Convert to GeoDataFrames
nodes_gdf, edges_gdf = ox.graph_to_gdfs(G)
print(f"\n✓ Converted to GeoDataFrames")
print(f"  Nodes GDF: {len(nodes_gdf)} rows")
print(f"  Edges GDF: {len(edges_gdf)} rows")

# Visualize the downloaded network
fig, ax = plt.subplots(figsize=(10, 15))

# Plot edges (roads)
edges_gdf.plot(ax=ax, linewidth=0.5, color='gray', alpha=0.7, label='Roads')

# Plot nodes (intersections)
nodes_gdf.plot(ax=ax, markersize=0.5, color='red', alpha=0.5, label='Nodes')

ax.set_title('Street Network', fontsize=16, fontweight='bold')
ax.set_xlabel('Longitude')
ax.set_ylabel('Latitude')
ax.legend()
ax.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

print("Network visualization complete!")

In [ ]:
# 4b. PATH-A: Save Network Data for Grid2Demand - Custom Parser

# Create output directory
output_dir = 'network_data'
os.makedirs(output_dir, exist_ok=True)

# Prepare node data with all required columns for grid2demand
node_data = pd.DataFrame({
    'node_id': range(len(nodes_gdf)),
    'osm_node_id': nodes_gdf.index,
    'x_coord': nodes_gdf.geometry.x,
    'y_coord': nodes_gdf.geometry.y,
    'longitude': nodes_gdf.geometry.x,
    'latitude': nodes_gdf.geometry.y,
    'activity_type': 'node',  # Required by grid2demand
    'is_boundary': 0,  # 0 = internal node, 1 = boundary node
    'zone_id': -1  # Will be assigned when zones are created
})

# Save node data
node_file = os.path.join(output_dir, 'node.csv')
node_data.to_csv(node_file, index=False)
print(f"✓ Saved node data: {node_file}")
print(f"  Total nodes: {len(node_data):,}")

# Create OSM to sequential node ID mapping
osm_to_seq = dict(zip(nodes_gdf.index, range(len(nodes_gdf))))

# Prepare link data with required columns
links = []
for idx, (u, v, key, data) in enumerate(G.edges(keys=True, data=True)):
    # Parse lanes - handle various formats
    lanes = data.get('lanes', 1)
    if isinstance(lanes, list):
        lanes = int(lanes[0]) if lanes else 1
    elif isinstance(lanes, str):
        try:
            lanes = int(lanes.split(';')[0])  # Take first value if multiple
        except:
            lanes = 1
    else:
        try:
            lanes = int(lanes)
        except:
            lanes = 1
    
    link = {
        'link_id': idx,
        'from_node_id': osm_to_seq[u],
        'to_node_id': osm_to_seq[v],
        'length': data.get('length', 0),  # in meters
        'lanes': lanes,
        'capacity': int(lanes * 1800),  # 1800 vehicles per hour per lane
        'free_speed': 35,  # default 35 mph
        'link_type': str(data.get('highway', 'residential')),
        'geometry': ''  # Optional: can add WKT geometry string
    }
    links.append(link)

link_data = pd.DataFrame(links)

# Save link data
link_file = os.path.join(output_dir, 'link.csv')
link_data.to_csv(link_file, index=False)
print(f"✓ Saved link data: {link_file}")
print(f"  Total links: {len(link_data):,}")

# Create a dummy POI file with all required columns for grid2demand
# Based on error, grid2demand expects: geometry, building, centroid, amenity
dummy_poi = pd.DataFrame({
    'poi_id': [0],
    'x_coord': [node_data['longitude'].mean()],
    'y_coord': [node_data['latitude'].mean()],
    'longitude': [node_data['longitude'].mean()],
    'latitude': [node_data['latitude'].mean()],
    'poi_type': ['placeholder'],
    'name': ['placeholder'],
    'area': [100.0],
    'zone_id': [-1],
    'geometry': ['POINT(0 0)'],  # WKT format
    'building': ['yes'],
    'centroid': ['POINT(0 0)'],
    'amenity': ['placeholder']
})
poi_file = os.path.join(output_dir, 'poi.csv')
dummy_poi.to_csv(poi_file, index=False)
print(f"✓ Created placeholder POI file: {poi_file}")

print(f"\n✓ Network data saved to: {output_dir}/")

In [ ]:
# 4a. PATH-B: Select, Download, and Save OSM Network Data for Grid2Demand Using osm2gmns Library

# Create output directory
output_dir = 'network_data_testing'
os.makedirs(output_dir, exist_ok=True)
osm_filepath = f"{output_dir}/map.osm"

# OSM Area ids
# search at: nominatim.openstreetmap.org
brooklyn_id = 369518
antwerp_id = 53114
brussels_id = 2404020
ghent_id = 897671
lviv_id = 2032280

# Download directly from OSM
print("Downloading OSM network...")
og.downloadOSMData(brooklyn_id, osm_filepath)


In [ ]:
# 4b. PATH-B: Open OSM, extract, convert, and save GMNS file

# Open OSM, convert to GMNS, and clean as desired
print("Loading and converting OSM network to GMNS...")
net = og.getNetFromFile(osm_filepath, mode_types='auto', POI=True)
og.fillLinkAttributesWithDefaultValues(net)
#og.consolidateComplexIntersections(net, auto_identify=True)

# Save to CSV files in GMNS format
og.outputNetToCSV(net, output_dir)

print(f"✓ Network downloaded, converted, and saved to GMNS format!")
print(f"  Nodes: {net.number_of_nodes:,}")
print(f"  Links: {net.number_of_links:,}")

In [ ]:
# 5. Initialize Grid2Demand Network - Now we'll use grid2demand to read the network and create zones

# Initialize grid2demand network
print("Initializing grid2demand network...\n")

# Create a GRID2DEMAND object (newer API)
net = gd.GRID2DEMAND(input_dir=output_dir)

# Load the network files
net.load_network()

print(f"✓ Network loaded successfully!")
print(f"  Nodes: {len(net.node_dict):,}")
print(f"  POIs: {len(net.poi_dict):,}")
# Note: link information is stored differently in grid2demand
if hasattr(net, 'link_dict'):
    print(f"  Links: {len(net.link_dict):,}")
elif hasattr(net, 'link_list'):
    print(f"  Links: {len(net.link_list):,}")

In [ ]:
# 6. Create Grid-Based Zones - Grid2demand will partition the area into grid-based zones (TAZs)

# Generate zone.csv from node by specifying number of x blocks and y blocks
#net.net2grid(num_x_blocks=10, num_y_blocks=10)
net.net2grid(cell_width=1, cell_height=1, unit="km")
print(f"  Zones in network: {len(net.zone_dict)}")

# Load zone from zone.csv
net.taz2zone()

# Map zones with nodes and poi, viseversa
net.map_zone_node_poi()

# Calculate zone-to-zone distance matrix
net.calc_zone_od_distance(pct=1.0)


In [ ]:
# 7. Calculate demand by running gravity model - save results (overwriting orig files)
net.run_gravity_model()

net.save_results_to_csv(zone_od_dist_matrix=True, demand_od_matrix=True, overwrite_file=True)

In [ ]:
# 8. Load and Analyze Demand Matrix

# Load the generated demand matrix
demand_file = os.path.join(output_dir, 'demand.csv')

if os.path.exists(demand_file):
    demand_df = pd.read_csv(demand_file)
    
    print("Demand Matrix Summary:")
    print("=" * 60)
    print(f"Total OD pairs: {len(demand_df):,}")
    print(f"Total trips: {demand_df['volume'].sum():,.0f}")
    print(f"\nTrip Volume Statistics:")
    print(f"  Mean:   {demand_df['volume'].mean():.2f}")
    print(f"  Median: {demand_df['volume'].median():.2f}")
    print(f"  Max:    {demand_df['volume'].max():.2f}")
    print(f"  Min:    {demand_df['volume'].min():.2f}")
    
    # Top 10 OD pairs by volume
    print(f"\nTop 10 OD Pairs by Volume:")
    print(demand_df.nlargest(10, 'volume')[['o_zone_id', 'd_zone_id', 'volume']])
else:
    print(f"Demand file not found: {demand_file}")
    print("Check output directory for generated files.")

In [ ]:
# 10. Visualize Zone Structure

# Load zone geometry if available
zone_file = os.path.join(output_dir, 'zone.csv')

if os.path.exists(zone_file):
    zone_df = pd.read_csv(zone_file)
    
    fig, ax = plt.subplots(figsize=(10, 15))
    
    # Plot the network
    edges_gdf.plot(ax=ax, linewidth=0.3, color='lightgray', alpha=0.5)
    
    # Plot zone centroids
    zone_gdf = gpd.GeoDataFrame(
        zone_df,
        geometry=gpd.points_from_xy(zone_df['x_coord'], zone_df['y_coord']),
        crs='EPSG:4326'
    )
    
    zone_gdf.plot(ax=ax, markersize=50, color='red', alpha=0.7, 
                  edgecolor='black', linewidth=1, label='Zone Centroids')
    
    # Add zone IDs
    for idx, row in zone_gdf.iterrows():
        ax.annotate(text=row['zone_id'], 
                   xy=(row.geometry.x, row.geometry.y),
                   fontsize=6, ha='center', va='center',
                   color='white', weight='bold')
    
    ax.set_title('Zone Structure', fontsize=16, fontweight='bold')
    ax.set_xlabel('Longitude')
    ax.set_ylabel('Latitude')
    ax.legend()
    ax.grid(True, alpha=0.3)
    
    plt.tight_layout()
    plt.show()
    
    print(f"\n✓ Zone visualization complete!")
    print(f"  Total zones: {len(zone_df)}")
else:
    print(f"Zone file not found: {zone_file}")